In [20]:
from pybaseball import schedule_and_record
import pandas as pd
import hvplot.pandas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import svm

In [2]:
mariners_data = schedule_and_record(2022, 'SEA')

/opt/anaconda3/lib/python3.9/site-packages/pybaseball/team_results.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('',1) #not a useful column
/opt/anaconda3/lib/python3.9/site-packages/pybaseball/team_results.py:83: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('Streak2',1)


In [3]:
mariners_data = mariners_data.set_index('Date')

In [4]:
mariners_data['Runs_Scored'] = mariners_data['R'].cumsum()

In [5]:
mariners_data['Runs_Allowed'] = mariners_data['RA'].cumsum()


In [6]:
mariners_data['RD'] = mariners_data['Runs_Scored'] - mariners_data['Runs_Allowed']


In [7]:
mariners_data = mariners_data[['Home_Away', 'W/L','W-L', 'R','RA', 'Rank', 'GB', 'D/N', 'Streak', 'Runs_Scored', 'Runs_Allowed', 'RD']]

In [10]:
mariners_data['W/L'] = mariners_data['W/L'].replace('W-wo', 'W')
mariners_data['W/L'] = mariners_data['W/L'].replace('L-wo', 'L')

In [11]:
mariners_data = mariners_data.drop(columns='GB')


In [12]:
#Replace W with 1 and L with 0
#Replace Home/Away @ with 0, Home with 1
#Replace D (Day) with 0 and N (night) with 1

mariners_data['W/L'] = mariners_data['W/L'].replace('W', '1')
mariners_data['W/L'] = mariners_data['W/L'].replace('L', '0')
mariners_data['Home_Away'] = mariners_data['Home_Away'].replace('@', '0')
mariners_data['Home_Away'] = mariners_data['Home_Away'].replace('Home', '1')
mariners_data['D/N'] = mariners_data['D/N'].replace('D', '0')
mariners_data['D/N'] = mariners_data['D/N'].replace('N', '1')

In [13]:
mariners_data['W/L'] = mariners_data['W/L'].astype(int)
mariners_data['Home_Away'] = mariners_data['Home_Away'].astype(int)
mariners_data['D/N'] = mariners_data['D/N'].astype(int)

In [14]:
#Calculate Cumulative Wins for the Season
mariners_data['Win_Total'] = mariners_data['W/L'].cumsum()

In [15]:
#Drop Wins - Losses Column
mariners_data = mariners_data.drop(columns='W-L')


In [16]:
y = mariners_data['W/L']
X = mariners_data.drop(columns='W/L')

In [25]:
mariners_data

,Home_Away,W/L,R,RA,Rank,D/N,Streak,Runs_Scored,Runs_Allowed,RD,Win_Total
Date,,,,,,,,,,,
"Friday, Apr 8",0,1,2.0,1.0,2.0,0,1,2.0,1.0,1.0,1
"Saturday, Apr 9",0,1,4.0,3.0,1.0,0,2,6.0,4.0,2.0,2
"Sunday, Apr 10",0,0,4.0,10.0,2.0,0,-1,10.0,14.0,-4.0,2
"Monday, Apr 11",0,0,0.0,4.0,2.0,1,-2,10.0,18.0,-8.0,2
"Tuesday, Apr 12",0,0,2.0,3.0,3.0,0,-3,12.0,21.0,-9.0,2
...,...,...,...,...,...,...,...,...,...,...,...
"Sunday, Oct 2",1,0,3.0,10.0,2.0,0,-1,666.0,603.0,63.0,87
"Monday, Oct 3",1,0,3.0,4.0,2.0,1,-2,669.0,607.0,62.0,87
"Tuesday, Oct 4 (1)",1,1,7.0,6.0,2.0,0,1,676.0,613.0,63.0,88
